# 1. Data Colletion

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("../datasets/rent_apartments.csv")

In [4]:
data.sample(5)

,address,area,constraction_year,rooms,bedrooms,bathrooms,balcony,storage,parking,furnished,garage,garden,energy,facilities,zip,neighborhood,rent
724,1012 VC Amsterdam (Hemelrijk),42.0,1906,2,1,1,no,no,no,no,no,Not present,NaN,"Shower, Toilet",1012 VC,Hemelrijk,1350
167,1017 BJ Amsterdam (Gouden Bocht),39.0,1905,2,1,1,no,no,no,yes,no,Not present,B,NaN,1017 BJ,Gouden Bocht,1650
622,1102 VK Amsterdam (Venserpolder West),109.0,1984,7,6,1,yes,yes,no,no,no,Not present,C,"Storage space, Toilet",1102 VK,Venserpolder West,2200
600,1082 XT Amsterdam (Gelderlandpleinbuurt),76.0,1960,2,1,1,yes,no,no,yes,no,Not present,NaN,NaN,1082 XT,Gelderlandpleinbuurt,1800
1294,1058 JN Amsterdam (Surinamepleinbuurt),68.0,1933,3,2,1,yes,no,no,no,no,Not present,D,"Bath, Toilet",1058 JN,Surinamepleinbuurt,1850


In [5]:
data.shape

(1728, 17)

# 2. Data Preparation

In [6]:
data.dtypes

address               object
area                 float64
constraction_year      int64
rooms                  int64
bedrooms               int64
bathrooms              int64
balcony               object
storage               object
parking               object
furnished             object
garage                object
garden                object
energy                object
facilities            object
zip                   object
neighborhood          object
rent                   int64
dtype: object

In [7]:
data_encoded = pd.get_dummies(data, 
                              columns=['balcony', 'parking', 'furnished', 'garage', 'storage'],
                              drop_first=True)

In [8]:
data_encoded.shape

(1728, 17)

In [9]:
data_encoded.garden.unique()

array(['Not present', 'Present (47 m²)', 'Present (29 m²)',
       'Present (75 m²)', 'Present (40 m², located on the north)',
       'Present (50 m²)', 'Present (20 m², located on the south)',
       'Present (1 m²)', 'Present (15 m²)', 'Present (25 m²)',
       'Present (12 m²)', 'Present (45 m², located on the south)',
       'Present (26 m², located on the south-east)',
       'Present (20 m², located on the north-east)',
       'Present (42 m², located on the west)', 'Present (46 m²)',
       'Present (45 m², located on the south-west)',
       'Present (60 m², located on the south-west)',
       'Present (50 m², located on the south)',
       'Present (40 m², located on the north-east)', 'Present (16 m²)',
       'Present (60 m²)', 'Present (65 m², located on the south)',
       'Present (90 m²)', 'Present (85 m²)',
       'Present (85 m², located on the south-west)',
       'Present (500 m², located on the west)',
       'Present (45 m², located on the west)',
       'Present (1

In [10]:
import re

In [11]:
re.findall(r'\d+', 'Present (47 m²)')

['47']

In [12]:
data_encoded['garden'] = data_encoded['garden'].apply(lambda x: int(re.findall(r'\d+', x)[0]) if x != 'Not present' else 0)

In [13]:
data_encoded.sample(5)

,address,area,constraction_year,rooms,bedrooms,bathrooms,garden,energy,facilities,zip,neighborhood,rent,balcony_yes,parking_yes,furnished_yes,garage_yes,storage_yes
151,1017 XR Amsterdam (Den Texbuurt),135.0,1890,3,2,2,0,B,Lift,1017 XR,Den Texbuurt,4000,True,True,True,True,False
249,1016 HP Amsterdam (Felix Meritisbuurt),90.0,1900,3,2,1,0,NaN,NaN,1016 HP,Felix Meritisbuurt,3000,False,False,True,False,False
1548,1107 CJ Amsterdam (Gein Zuidoost),55.0,1984,2,1,1,0,B,Storage space,1107 CJ,Gein Zuidoost,1200,False,True,True,False,True
467,1011 NA Amsterdam (Uilenburg),90.0,2014,3,2,2,0,A,"Cable TV, Internet connection, Swimming pool, ...",1011 NA,Uilenburg,2600,True,True,True,True,False
140,1017 NH Amsterdam (Leidsebuurt Noordoost),40.0,1940,2,1,1,0,NaN,NaN,1017 NH,Leidsebuurt Noordoost,1450,False,False,True,False,False


In [14]:
data_encoded.garden.unique()

array([  0,  47,  29,  75,  40,  50,  20,   1,  15,  25,  12,  45,  26,
        42,  46,  60,  16,  65,  90,  85, 500,  30,  49,  51,  80,  27,
        56,   9, 200,  32, 100,  34])

# 3. Model Building

## 3.1. Defining X and y

In [15]:
columns = ['area', 
            'constraction_year', 
            'bedrooms', 
            'garden', 
            'balcony_yes', 
            'parking_yes', 
            'furnished_yes', 
            'garage_yes', 
            'storage_yes']
X = data_encoded[columns]

y = data_encoded['rent'] 

## 3.2. Split the dataset

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## 3.3. Model Building

In [18]:
from sklearn.ensemble import RandomForestRegressor

In [19]:
rf = RandomForestRegressor()
rf.fit(X_train, y_train)

,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",100
,"criterion criterion: {""squared_error"", ""absolute_error"", ""friedman_mse"", ""poisson""}, default=""squared_error""The function to measure the quality of a split. Supported criteriaare ""squared_error"" for the mean squared error, which is equal tovariance reduction as feature selection criterion and minimizes the L2loss using the mean of each terminal node, ""friedman_mse"", which usesmean squared error with Friedman's improvement score for potentialsplits, ""absolute_error"" for the mean absolute error, which minimizesthe L1 loss using the median of each terminal node, and ""poisson"" whichuses reduction in Poisson deviance to find splits.Training using ""absolute_error"" is significantly slowerthan when using ""squared_error""... versionadded:: 0.18 Mean Absolute Error (MAE) criterion... versionadded:: 1.0 Poisson criterion.",'squared_error'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",None
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=1.0The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None or 1.0, then `max_features=n_features`... note:: The default of 1.0 is equivalent to bagged trees and more randomness can be achieved by setting smaller values, e.g. 0.3... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to 1.0.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",1.0
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsample

## 3.4. Evaluate Model

In [20]:
rf.score(X_test, y_test)

0.7290712825378436

## 3.5. Predicting

In [21]:
rf.predict([[85, 2015, 2, 20, 1, 1, 0, 0, 1]])

/home/u631013/working-directory/ml-in-production/.env/lib/python3.12/site-packages/sklearn/utils/validation.py:2691: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


array([1958.86666667])

## 3.6. Tuning Hyperparameters

In [22]:
from sklearn.model_selection import GridSearchCV

In [23]:
grid_space = {'n_estimators': [100, 200, 300], 
              'max_depth': [3, 6, 9, 12]}


In [24]:
grid = GridSearchCV(RandomForestRegressor(), 
                    param_grid=grid_space, 
                    cv=5, 
                    scoring = 'r2')

In [25]:
model_grid = grid.fit(X_train, y_train)

In [26]:
print(f'Best hyperparameters are {model_grid.best_params_}, score = {model_grid.best_score_}')

Best hyperparameters are {'max_depth': 9, 'n_estimators': 300}, score = 0.7144458350521737


In [27]:
model_grid.best_estimator_.predict([[85, 2015, 2, 20, 1, 1, 0, 0, 1]])

/home/u631013/working-directory/ml-in-production/.env/lib/python3.12/site-packages/sklearn/utils/validation.py:2691: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


array([1991.7685887])

In [28]:
rf_best = model_grid.best_estimator_

# 4. Models Management

In [29]:
import pickle as pk

In [30]:
pk.dump(rf_best, open('../models/rf_best.pkl', 'wb'))

In [31]:
! ls ../models/

rf_best.pkl


In [32]:
rf_model = pk.load(open('../models/rf_best.pkl', 'rb'))

In [33]:
rf_model.predict([[85, 2015, 2, 20, 1, 1, 0, 0, 1]])

/home/u631013/working-directory/ml-in-production/.env/lib/python3.12/site-packages/sklearn/utils/validation.py:2691: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


array([1991.7685887])